In [1]:
from dotenv import load_dotenv
# Load environment variables from .env file

import os
load_dotenv()
OPENAI_API_KEY = os.getenv("openai_key")

In [39]:
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

# --- Initialize the LLM ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY)

# --- Define your tools ---
def add_numbers(a: float, b: float) -> float:
    """Add two numbers and return the sum."""
    return a + b

def get_weather(city: str) -> str:
    """Pretend to get the weather (demo mode)."""
    return f"The weather in {city} is sunny ☀️ (demo mode)."

# Wrap tools for LangChain
tools = [
    Tool(
        name="Add Numbers",
        func=lambda x: add_numbers(*[float(i) for i in x.split()]),
        description="Add two numbers provided as input, separated by a space. Example: '2.5 7.3'"
    ),
    Tool(
        name="Get Weather",
        func=get_weather,
        description="Get the weather for a given city name."
    )
]


# --- Initialize the agent ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [40]:
query = "Hi! Can you tell me the weather in Paris and add 2.5 + 7.3?"
response = agent.invoke(query)
print("\n💬 Agent Answer:\n", response["output"])




> Entering new AgentExecutor chain...
I need to get the weather for Paris and also perform the addition of the numbers 2.5 and 7.3. I'll start with the weather request.

Action: Get Weather  
Action Input: "Paris"  
Observation: The weather in Paris is sunny ☀️ (demo mode).
Thought:I have obtained the weather information for Paris. Now, I will proceed to add the numbers 2.5 and 7.3.

Action: Add Numbers  
Action Input: "2.5 7.3"  
Observation: 9.8
Thought:I now know the final answer, which includes both the weather in Paris and the result of the addition. 

Final Answer: The weather in Paris is sunny ☀️ and 2.5 + 7.3 equals 9.8.

> Finished chain.

💬 Agent Answer:
 The weather in Paris is sunny ☀️ and 2.5 + 7.3 equals 9.8.


In [42]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import requests

def simple_duckduckgo_search(query: str) -> str:
    """Stable fallback web search using DuckDuckGo Instant Answer API."""
    try:
        resp = requests.get(
            "https://api.duckduckgo.com/",
            params={"q": query, "format": "json", "no_html": 1, "no_redirect": 1},
            timeout=10
        )
        data = resp.json()
        if data.get("AbstractText"):
            return data["AbstractText"]
        elif data.get("RelatedTopics"):
            results = [t.get("Text") for t in data["RelatedTopics"] if "Text" in t]
            return "\n".join(results[:3])
        else:
            return "No relevant results found."
    except Exception as e:
        return f"Search error: {e}"

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=simple_duckduckgo_search,
        description="Look up people or topics on the web."
    )
]

# --- create agent with tool ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# --- ask the question ---
query = "Who is Ada Lovelace?"
response = agent.invoke(query)

print("\n💬 Agent Answer:\n", response["output"])




> Entering new AgentExecutor chain...
I need to gather information about Ada Lovelace, her contributions, and significance in history. 
Action: DuckDuckGo Search
Action Input: "Who is Ada Lovelace?"
Observation: Augusta Ada King, Countess of Lovelace, also known as Ada Lovelace, was an English mathematician and writer chiefly known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. She was the first to recognise that the machine had applications beyond pure calculation. Ada Lovelace is sometimes considered to be the first computer programmer. Ada Lovelace was the only legitimate child of poet Lord Byron and reformer Anne Isabella Milbanke. All her half-siblings, Lord Byron's other children, were born out of wedlock to other women. Lord Byron separated from his wife a month after Ada was born and left England forever. He died in Greece whilst fighting in the Greek War of Independence, when she was eight. Lady Byron was anxious about 